In [ ]:
import numpy as np
import math
import pickle
import warnings

import SE
import SO
from util import generate_groups, generate_non_overlap, generate_means
from Grouped_Bandit import gb

## Experiment - Investigation of effect of gap value $\Delta$
- We consider how the total sample compleixty (represented by total arm pulls) and accuracy of the algorithms outlined are affected by the value of $\Delta$.

In [ ]:
# Adjust model variable
c_val=1
eta=0.01
num_groups=10
num_arms=100
thy=0
gap_list=[0.1, 0.2, 0.4]

# Adjust experiment variable
num_trials=10
num_seeds=10

result = dict()
for seed_i in range(num_seeds):
    # Set seed
    seed = seed_i
    np.random.seed(seed)
    
    gaps_info = {}
    for gap in gap_list:
        groups = generate_groups(num_groups, num_arms)
        means = generate_means(num_groups, num_arms, gap, groups)
        gb1 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=groups, means=means)
        gb2 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=gb1.groups, means=gb1.true_means)

        # Record experimental results
        succ_count_list_SE = []
        succ_count_list_SO = []
        arm_pulls_list_SE = []
        arm_pulls_list_SO = []
        for trial_idx in range(num_trials):
            
            G_final = SE.succ_elim(gb1, thy)
            
            ## 1 if the result is correct, 0 otherwise
            if set(G_final).issubset(set(gb1.find_true_objectives())):
                succ_count_list_SE.append(1)
            else:
                succ_count_list_SE.append(0)

            arm_pulls_list_SE.append(gb1.compute_total_arm_pulls())
            print(gb1.compute_total_arm_pulls())
            gb1 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=gb1.groups, means=gb1.true_means)
        print('SE done')
        for trial_idx in range(num_trials):
            G_final_2 = SO.stable_opt(gb2, c_val, eta)
            if set(G_final_2).issubset(set(gb2.find_true_objectives())):
                succ_count_list_SO.append(1)
            else:
                succ_count_list_SO.append(0)

            arm_pulls_list_SO.append(gb2.compute_total_arm_pulls())
            print(gb2.compute_total_arm_pulls())
            gb2 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=gb2.groups, means=gb2.true_means)
        succ_rate_SE = sum(succ_count_list_SE) / num_trials
        succ_rate_SO = sum(succ_count_list_SO) / num_trials
        avg_num_pulls_SE = sum(arm_pulls_list_SE) / num_trials
        print(f'{gap}, SE: {succ_rate_SE}, SO: {succ_rate_SO}, pulls: {avg_num_pulls_SE}')
        gaps_info[gap] = {'succ_count_list_SE': succ_count_list_SE, 'succ_count_list_SO': succ_count_list_SO, 
                             'arm_pulls_list_SE': arm_pulls_list_SE, 'arm_pulls_list_SO': arm_pulls_list_SO}

    result[seed] = gaps_info

### Save into \Results folder
with open(f'Results/gap_value_c={c_val}_conf={eta}.pickle', 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Experiment - Comparison to the Naive approach
- In our implementation, we consider a comparison between the SE algorithm and the group-wise naive UCB approach discussed in our paper.

In [ ]:
# Adjust model variable
c_val=1
eta=0.01
num_groups=10
num_arms=100
thy=0
gap_list=[0.1, 0.2, 0.4]

# Adjust experiment variable
num_trials=10
num_seeds=10

naive_result_non_overlap = dict()
for seed_i in range(num_seeds):
    # Set seed
    seed = seed_i
    np.random.seed(seed)
    
    gaps_info = {}
    for gap in gap_list:
        groups = generate_non_overlap(num_groups,num_arms)
        means = generate_means(num_groups,num_arms,gap,groups)
        gb1 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=groups, means=means)
        gb2 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=gb1.groups, means=gb1.true_means)

        # Store emperimental results
        succ_count_list_SE = []
        succ_count_list_naive = []
        arm_pulls_list_SE = []
        arm_pulls_list_naive = []
        for trial_idx in range(num_trials):
            G_final = SE.succ_elim(gb1, thy)
            
            ## 1 if the result is correct, 0 otherwise
            if set(G_final).issubset(set(gb1.find_true_objectives())):
                succ_count_list_SE.append(1)
            else:
                succ_count_list_SE.append(0)

            arm_pulls_list_SE.append(gb1.compute_total_arm_pulls())
            print(gb1.compute_total_arm_pulls())
            gb1 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=gb1.groups, means=gb1.true_means)
        print('SE done')
        for trial_idx in range(num_trials):
            G_final_2 = Naive_UCB.group_wise_UCB(gb2, c_val, eta)
            if set(G_final_2).issubset(set(gb2.find_true_objectives())):
                succ_count_list_naive.append(1)
            else:
                succ_count_list_naive.append(0)

            arm_pulls_list_naive.append(gb2.compute_total_arm_pulls())
            print(gb2.compute_total_arm_pulls())
            gb2 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=gb2.groups, means=gb2.true_means)
        succ_rate_SE = sum(succ_count_list_SE) / num_trials
        succ_rate_naive = sum(succ_count_list_naive) / num_trials
        print(f'{gap}, SE: {succ_rate_SE}, naive:{succ_rate_naive}')
        gaps_info[gap] = {'succ_count_list_SE': succ_count_list_SE, 'succ_count_list_naive': succ_count_list_naive, 
                             'arm_pulls_list_SE': arm_pulls_list_SE, 'arm_pulls_list_naive': arm_pulls_list_naive}

    naive_result_non_overlap[seed] = gaps_info

### Save into \Results folder  
with open(f'Results/naive_non_overlap_c={c_val}_conf={eta}.pickle', 'wb') as handle:
    pickle.dump(naive_result_non_overlap, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Experiment - Different Bound Expressions (Theoretical vs Simplified choice)

In [ ]:
# Adjust model variable
c_val=1
eta=0.01
num_groups=10
num_arms=100
gap_list=[0.1, 0.2, 0.4]

# Adjust experiment variable
num_trials=10
num_seeds=10

result={}
for gap in gap_list:
    
    gaps_info= {}
    for seed in range(num_seeds):
        # Set seed
        np.random.seed(seed)
        groups = generate_groups(num_groups, num_arms)
        means = generate_means(num_groups, num_arms, gap, groups)
        gb1 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=groups, means=means)
        gb2 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=gb1.groups, means=gb1.true_means)
        
        # Store experimental results
        succ_count_list_SE = []
        succ_count_list_Theoretical = []
        arm_pulls_list_SE = []
        arm_pulls_list_Theoretical = []
        for trial_idx in range(num_trials):
            G_final = SE.succ_elim(gb1, thy=0)
            if set(G_final).issubset(set(gb1.find_true_objectives())):
                succ_count_list_SE.append(1)
            else:
                succ_count_list_SE.append(0)

            arm_pulls_list_SE.append(gb1.compute_total_arm_pulls())
            print(gb1.compute_total_arm_pulls())
            gb1 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=gb1.groups, means=gb1.true_means)
        print('done')
        for trial_idx in range(num_trials):
            G_final_2 = SE.succ_elim(gb2, thy=1)
            if set(G_final_2).issubset(set(gb2.find_true_objectives())):
                succ_count_list_Theoretical.append(1)
            else:
                succ_count_list_Theoretical.append(0)

            arm_pulls_list_Theoretical.append(gb2.compute_total_arm_pulls())
            print(gb2.compute_total_arm_pulls())
            gb2 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=gb2.groups, means=gb2.true_means)
        succ_rate_SE = sum(succ_count_list_SE) / num_trials
        succ_rate_Theoretical = sum(succ_count_list_Theoretical) / num_trials
        print(f'{seed}, SE: {succ_rate_SE}, Simple: {succ_rate_Theoretical}')
        gaps_info[seed] = {'succ_count_list_SE': succ_count_list_SE, 'succ_count_list_Theoretical': succ_count_list_Theoretical, 
                              'arm_pulls_list_SE': arm_pulls_list_SE, 'arm_pulls_list_Theoretical': arm_pulls_list_Theoretical}
    result[gap] = gaps_info
    
import pickle
with open(f'Results/bound_c={c_val}.pickle', 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Experiment - Simple Regret for different algorithms stopping criterions and gap values

### StableOpt Functions
You have the following choices of optimizer selection criterion for StableOpt:
1. `gb2_minmax_mean`: current group with highest worst-case mean
2. `gb2_minmax_lcb`: current group with highest worst-case LCB value
3. `gb2_history_lcb`: the group with highest worst-case LCB value from the start of algorithm run to current iteration

In [ ]:
# Adjust model variable
c_val=1
eta=0.01
num_groups=10
num_arms=100
thy=0
gap_list=[0.1, 0.2, 0.4]

# Adjust experiment variable
num_trials=100
seed=0

# Choice of stopping criterion (1 if the result is recorded in the dt)
minmax_mean=1
minmax_lcb=1
history_lcb=1

result = {}

for gap in gap_list:
    # Set seed
    np.random.seed(seed)
    
    # Initialize base instance
    groups = generate_groups(num_groups, num_arms)
    means = generate_means(num_groups, num_arms, gap, groups)
    gb0 = gb(num_groups=num_groups, num_arms=num_arms, c=c_val, gap=gap, dist='bernoulli', groups=groups, means=means)
    
    count = 0
    gap_result = {}

    for k in range(num_trials):
        gb1 = gb(num_groups=num_groups, num_arms=num_arms, groups=gb0.groups, means=gb0.true_means, c=c_val, gap=gap) # instance for SE
        gb2 = gb(num_groups=num_groups, num_arms=num_arms, groups=gb0.groups, means=gb0.true_means, c=c_val, gap=gap) # instance for StableOpt

        # param for StableOpt
        G_list = []
        curr_list = []
        history_best_l = -1 ## history best lcb candidate
        history_best_lcb = 0

        # param for SE
        m = gb1.groups
        c = np.arange(gb1.num_groups)

        ## instance data (we include all )
        true_optimal_group = gb0.find_true_objectives()
        true_optimal_min = min(gb0.true_means[gb1.groups[true_optimal_group][0]])
        result['true_optimal_group'] = true_optimal_group
        result['true_optimal_min'] = true_optimal_min
        
        gb1_summary = {}
        gb1_summary['gb1_minmax_lcb_val'] = []
        gb1_summary['gb1_minmax_lcb_G'] = []
        gb1_summary['gb1_minmax_lcb_regret'] = []
        gb1_summary['pulls'] = []
        
        gb2_summary = {}
        if minmax_mean:
            gb2_summary['gb2_minmax_mean_val'] = []
            gb2_summary['gb2_minmax_mean_G'] = []
            gb2_summary['gb2_minmax_mean_regret'] = []
        if minmax_lcb:
            gb2_summary['gb2_minmax_lcb_val'] = []
            gb2_summary['gb2_minmax_lcb_G'] = []
            gb2_summary['gb2_minmax_lcb_regret'] = []
        if history_lcb:
            gb2_summary['gb2_history_lcb_val'] = []
            gb2_summary['gb2_history_lcb_G'] = []
            gb2_summary['gb2_history_lcb_regret'] = []
        
        while c.size > 1:

            # One iteration for SE (begin)
            m = SE.compute_m_t(c, m, gb1, thy)
            c = SE.compute_c_t(c, m, gb1, thy)
            a = SE.compute_a_t(c, m)

            pulls = int(gb1.compute_total_arm_pulls() - gb2.compute_total_arm_pulls())
            if pulls > 0 and gb1.iter:
                lcb_t = gb1.empirical_means - (c_val / np.sqrt(gb1.individual_arm_pulls))
                
                gb1_minmax_lcb_val = np.array([lcb_t[g].min() for g in gb1.groups]).max()
                gb1_minmax_lcb_Gs = np.argwhere([lcb_t[g].min() for g in gb1.groups] == gb1_minmax_lcb_val).reshape(1,-1)[0]
                gb1_minmax_lcb_G = np.random.choice(gb1_minmax_lcb_Gs, 1)
                gb1_minmax_lcb_G_mean = gb1.true_means[gb1.groups[gb1_minmax_lcb_G][0]].min()

                gb1_summary['gb1_minmax_lcb_val'].append(gb1_minmax_lcb_G_mean)
                gb1_summary['gb1_minmax_lcb_G'].append(gb1_minmax_lcb_G)
                gb1_summary['gb1_minmax_lcb_regret'].append(true_optimal_min - gb1_minmax_lcb_G_mean)
                gb1_summary['pulls'].append(gb1.compute_total_arm_pulls())

            # One iteration for StableOpt with num of pulls equal to SE's one iteration
            for i in range(pulls) :
                ucb_t = gb2.empirical_means + (c_val / np.sqrt(gb2.individual_arm_pulls))
                lcb_t = gb2.empirical_means - (c_val / np.sqrt(gb2.individual_arm_pulls))

                G_t = SO.select_G_t(gb2, ucb_t)
                a_t = SO.select_a_t(gb2, lcb_t, G_t)

                if i == (pulls - 1):
                    gb2_minmax_mean_val = np.array([gb2.empirical_means[g].min() for g in gb2.groups]).max()
                    gb2_minmax_mean_G = np.argwhere([gb2.empirical_means[g].min() for g in gb2.groups] == gb2_minmax_mean_val).reshape(1,-1)[0]
                    gb2_minmax_mean_G_mean = gb2.true_means[gb2.groups[gb2_minmax_mean_G][0]].min()
                    gb2_minmax_lcb_val = np.array([lcb_t[g].min() for g in gb2.groups]).max()
                    gb2_minmax_lcb_G = np.argwhere([lcb_t[g].min() for g in gb2.groups] == gb2_minmax_lcb_val).reshape(1,-1)[0]
                    gb2_minmax_lcb_G_mean = gb2.true_means[gb2.groups[gb2_minmax_lcb_G][0]].min()

                history_best_l_mean = gb2.true_means[gb2.groups[history_best_l]].min()
                if (lcb_t[gb2.groups[G_t]].min() > history_best_lcb):
                    history_best_l = G_t
                    history_best_lcb = lcb_t[gb2.groups[G_t]].min()
                    history_best_l_mean = gb2.true_means[gb2.groups[history_best_l]].min()
                gb2.one_iteration(np.array([a_t]))

            # There are 3 choices for regret comparison: gb2_minmax_mean, gb2_minmax_lcb, history_best_lcb
            if pulls > 0:
                ## gb2_minmax_mean
                gb2_summary['gb2_minmax_mean_val'].append(gb2_minmax_mean_G_mean)
                gb2_summary['gb2_minmax_mean_G'].append(gb2_minmax_mean_G)
                gb2_summary['gb2_minmax_mean_regret'].append(true_optimal_min - gb2_minmax_mean_G_mean)

                ## gb2_minmax_lcb
                gb2_summary['gb2_minmax_lcb_val'].append(gb2_minmax_lcb_G_mean)
                gb2_summary['gb2_minmax_lcb_G'].append(gb2_minmax_lcb_G)
                gb2_summary['gb2_minmax_lcb_regret'].append(true_optimal_min - gb2_minmax_lcb_G_mean)

                ## history_best_lcb
                gb2_summary['gb2_history_lcb_val'].append(history_best_l_mean)
                gb2_summary['gb2_history_lcb_G'].append(history_best_l)
                gb2_summary['gb2_history_lcb_regret'].append(true_optimal_min - history_best_l_mean)

    #         print(gb1_summary, gb2_summary)
            # One iteration for SE (end)
            gb1.one_iteration(a)
            if np.array_equal(np.tile(a, (len(c), 1)), m[c]): # Check if all m_i^G have identical elements
                print('Multiple optimal groups found')
                break
            if (gb1.iter > 100000):
                print(f'bad: {m, c, a}')
                break
        print(f'{k}: {c} // {gb2_minmax_lcb_G}')

        gap_result[k] = {'gb1_summary': gb1_summary, 'gb2_summary': gb2_summary}

    result[gap] = gap_result

with open(f'Results/regret_result_c={c_val}.pickle', 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)